In [1]:
import os
import random
import numpy as np

import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from nasbench_nlp.nas_environment import Environment
from neural_predictor import regressor
from input_preprocessing import preprocess_nasbench_nlp

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

### Test on New Benchmark Datasets (NLP)

In [3]:
precomputed_logs_path = 'nasbench_nlp/train_logs_single_run/'

In [4]:
env = Environment(precomputed_logs_path)

In [5]:
search_set = env.get_precomputed_recepies()

In [6]:
LOOPS = 60
MAX_SAMPLES = 500
MAX_TIME_BUDGET = 8e5

### Oracle

In [7]:
val_loss, test_loss = [], []
    
env.reset()
for s in search_set:
    env.simulated_train(s, 50)
    if 'OK' == env.get_model_status(s):
        stats = env.get_model_stats(s, 49)
        val_loss.append(stats['val_loss'])
        test_loss.append(stats['test_loss'])

In [8]:
print(np.min(val_loss), np.std(val_loss))
print(np.min(test_loss), np.std(test_loss))

4.41530952065678 0.5854086570687619
4.368057399120145 0.5847845925439688


In [9]:
oracle = {'val_loss': val_loss, 'test_loss': test_loss}

### Random Search

In [10]:
env.reset()

random_val_avg, random_test_avg = [], []
random_val_std, random_test_std = [], []

val_loss, test_loss = [], []
for _ in tqdm(range(MAX_SAMPLES)):
    loop_val, loop_test = [], []
    for _ in range(LOOPS):
        idx = np.random.choice(len(search_set), 1, replace=False)[0]
        s = search_set[idx]

        env.simulated_train(s, 50)
        if 'OK' == env.get_model_status(s):
            stats = env.get_model_stats(s, 49)
            loop_val.append(stats['val_loss'])
            loop_test.append(stats['test_loss'])

    val_loss.append(np.mean(loop_val))
    test_loss.append(np.mean(loop_test))
    
    random_val_avg.append(np.min(val_loss))
    random_val_std.append(np.std(loop_val))
    
    random_test_avg.append(np.min(test_loss))
    random_test_std.append(np.std(loop_test))

  0%|          | 0/500 [00:00<?, ?it/s]

In [11]:
np.save('outputs/nlp_random_val_avg_by_samples.npy', random_val_avg)
np.save('outputs/nlp_random_val_std_by_samples.npy', random_val_std)
np.save('outputs/nlp_random_test_avg_by_samples.npy', random_test_avg)
np.save('outputs/nlp_random_test_std_by_samples.npy', random_test_std)

In [ ]:
random_val_avg, random_test_avg = [], []
random_val_std, random_test_std = [], []

val_loss, test_loss = [], []
for budget in tqdm(range(0, int(MAX_TIME_BUDGET), 1600)): # 500 loops
    loop_val, loop_test = [], []

    for _ in range(LOOPS):
        while True:
            time_spent = 0
            idx = np.random.choice(len(search_set), 1, replace=False)[0]
            s = search_set[idx]

            env.simulated_train(s, 50)
            if 'OK' == env.get_model_status(s):
                stats = env.get_model_stats(s, 49)
                loop_val.append(stats['val_loss'])
                loop_test.append(stats['test_loss'])
                time_spent += stats['wall_time']
                if time_spent > budget:
                    break
    
    val_loss.append(np.mean(loop_val))
    test_loss.append(np.mean(loop_test))
    
    random_val_avg.append(np.min(val_loss))
    random_val_std.append(np.std(loop_val))
    
    random_test_avg.append(np.min(test_loss))
    random_test_std.append(np.std(loop_test))

  0%|          | 0/500 [00:00<?, ?it/s]

In [13]:
np.save('outputs/nlp_random_val_avg_by_time.npy', random_val_avg)
np.save('outputs/nlp_random_val_std_by_time.npy', random_val_std)
np.save('outputs/nlp_random_test_avg_by_time.npy', random_test_avg)
np.save('outputs/nlp_random_test_std_by_time.npy', random_test_std)

### Neural Predictor

In [15]:
# number of nodes <= 24, number of hidden states <= 3, number of linear input vectors n <= 3

In [16]:
# val_loss, test_loss = [], []

# env.reset()
# val_loss_i, test_loss_i = [], []
# for K in range(2000-172):
#     K = k + 1
#     idx = np.random.choice(len(search_set), 172+K, replace=False)[0]

#     s = search_set[idx]
#     env.simulated_train(s, 50)
    
#     if 'OK' == env.get_model_status(s):
#         stats = env.get_model_stats(s, 49)
#         val_loss_i.append(stats['val_loss'])
#         test_loss_i.append(stats['test_loss'])
        
#     val_loss.append(np.min(val_loss_i))
#     test_loss.append(np.min(test_loss_i))

In [17]:
N = 172
def get_N_samples(N):
    env.reset()
    
    models = []
    stats = []

    for _ in range(N):
        while True:
            idx = np.random.choice(len(search_set), 1, replace=False)[0]
            model = search_set[idx]
            
            if model not in models:
                env.simulated_train(model, 50)
                
                if 'OK' == env.get_model_status(model):   
                    models.append(model)
                    stats.append(env.get_model_stats(model, 49))
                    break
                
    return preprocess_nasbench_nlp(models, stats)

In [18]:
# training part
train_data = get_N_samples(N)
reg = regressor([train_data['X'], train_data['norm_A'], train_data['norm_AT']], train_data['val_loss'], mode='loss')

In [19]:
# testing part
np_val_avg, np_test_avg = [], []
np_val_std, np_test_std = [], []

val_acc, test_acc = [], []
for k in tqdm(range(MAX_SAMPLES - N)):
    K = k + 1

    loop_val, loop_test = [], []
    for _ in range(LOOPS):
        test_models = get_N_samples(N+K)
        pred_acc = reg.predict([test_models['X'], test_models['norm_A'], test_models['norm_AT']]).ravel()
        
        topk_idx = tf.math.top_k(-pred_acc, k=K).indices.numpy()
        selected_val = test_models['val_loss'][topk_idx]
        selected_test = test_models['test_loss'][topk_idx]
        
        best_val_idx = np.argmin(selected_val)
        
        loop_val.append(selected_val[best_val_idx])
        loop_test.append(selected_test[best_val_idx])

    val_acc.append(np.mean(loop_val))
    test_acc.append(np.mean(loop_test))
    
    np_val_avg.append(np.min(val_acc))
    np_val_std.append(np.std(loop_val))
    
    np_test_avg.append(np.min(test_acc))
    np_test_std.append(np.std(loop_test))

  0%|          | 0/328 [00:00<?, ?it/s]

In [31]:
np.save('outputs/nlp_np_val_avg_by_samples.npy', np_val_avg)
np.save('outputs/nlp_np_val_std_by_samples.npy', np_val_std)
np.save('outputs/nlp_np_test_avg_by_samples.npy', np_test_avg)
np.save('outputs/nlp_np_test_std_by_samples.npy', np_test_std)

In [ ]:
np_val_avg, np_test_avg = [], []
np_val_std, np_test_std = [], []

val_acc, test_acc = [], []
for budget in tqdm(range(int(train_time), int(MAX_TIME_BUDGET), 1600)): # 5000 loops
        
    loop_val, loop_test = [], []
    for _ in range(LOOPS):
        time_spent = 0
        test_models = get_N_samples(N+budget//100)
        
        for i in range(len(test_models['times'])):
            time_spent = time_spent + test_models['times'][i]
            if time_spent >= budget:
                break
        
        X = test_models['X'][:i]
        A = test_models['norm_A'][:i]
        AT = test_models['norm_AT'][:i]
        labels = test_models['val_acc'][:i]
        test_labels = test_models['test_acc'][:i]
        
        pred_acc = reg.predict([X, A, AT]).ravel()
        
        selected_val = labels
        selected_test = test_labels
        
        best_val_idx = np.argmax(selected_val)
        
        loop_val.append(selected_val[best_val_idx])
        loop_test.append(selected_test[best_val_idx])

    val_acc.append(np.mean(loop_val))
    test_acc.append(np.mean(loop_test))
    
    np_val_avg.append(np.max(val_acc))
    np_val_std.append(np.std(loop_val))
    
    np_test_avg.append(np.max(test_acc))
    np_test_std.append(np.std(loop_test))